## Embeddings

Sa ating nakaraang halimbawa, gumamit tayo ng high-dimensional na bag-of-words vectors na may habang `vocab_size`, at tahasan nating kino-convert ang low-dimensional positional representation vectors sa sparse one-hot representation. Ang one-hot representation na ito ay hindi memory-efficient. Bukod dito, ang bawat salita ay itinuturing na hiwalay sa isa't isa, ibig sabihin, ang one-hot encoded vectors ay hindi nagpapakita ng anumang semantikong pagkakatulad sa pagitan ng mga salita.

Sa yunit na ito, ipagpapatuloy natin ang pag-explore sa **News AG** dataset. Upang magsimula, i-load natin ang data at kunin ang ilang mga depinisyon mula sa nakaraang notebook.


In [1]:
import torch
import torchtext
import numpy as np
from torchnlp import *
train_dataset, test_dataset, classes, vocab = load_dataset()
vocab_size = len(vocab)
print("Vocab size = ",vocab_size)

Loading dataset...


d:\WORK\ai-for-beginners\5-NLP\14-Embeddings\data\train.csv: 29.5MB [00:01, 18.8MB/s]                            
d:\WORK\ai-for-beginners\5-NLP\14-Embeddings\data\test.csv: 1.86MB [00:00, 11.2MB/s]                          


Building vocab...
Vocab size =  95812


## Ano ang embedding?

Ang ideya ng **embedding** ay ang pagrepresenta ng mga salita gamit ang mas mababang-dimensional na dense vectors, na sa isang paraan ay nagpapakita ng semantikong kahulugan ng isang salita. Tatalakayin natin sa susunod kung paano bumuo ng makabuluhang word embeddings, ngunit sa ngayon, isipin muna natin ang embeddings bilang isang paraan upang bawasan ang dimensionality ng isang word vector.

Kaya, ang embedding layer ay tatanggap ng isang salita bilang input, at maglalabas ng output vector na may tinukoy na `embedding_size`. Sa isang paraan, ito ay halos katulad ng `Linear` layer, ngunit sa halip na tumanggap ng one-hot encoded vector, ito ay makakatanggap ng word number bilang input.

Sa paggamit ng embedding layer bilang unang layer sa ating network, maaari tayong lumipat mula sa bag-of-words patungo sa **embedding bag** model, kung saan una nating kino-convert ang bawat salita sa ating teksto sa kaukulang embedding, at pagkatapos ay kinakalkula ang isang aggregate function sa lahat ng mga embeddings na iyon, tulad ng `sum`, `average`, o `max`.

![Larawan na nagpapakita ng isang embedding classifier para sa limang sequence na salita.](../../../../../translated_images/tl/embedding-classifier-example.b77f021a7ee67eee.webp)

Ang ating classifier neural network ay magsisimula sa embedding layer, pagkatapos ay aggregation layer, at linear classifier sa ibabaw nito:


In [2]:
class EmbedClassifier(torch.nn.Module):
    def __init__(self, vocab_size, embed_dim, num_class):
        super().__init__()
        self.embedding = torch.nn.Embedding(vocab_size, embed_dim)
        self.fc = torch.nn.Linear(embed_dim, num_class)

    def forward(self, x):
        x = self.embedding(x)
        x = torch.mean(x,dim=1)
        return self.fc(x)

### Pagtugon sa Iba't Ibang Laki ng Sequence ng Variable

Dahil sa arkitekturang ito, kailangang likhain ang mga minibatch para sa ating network sa isang tiyak na paraan. Sa nakaraang unit, noong ginagamit ang bag-of-words, lahat ng BoW tensors sa isang minibatch ay may pantay na laki na `vocab_size`, anuman ang aktwal na haba ng ating text sequence. Kapag lumipat na tayo sa word embeddings, magkakaroon tayo ng iba't ibang bilang ng mga salita sa bawat text sample, at kapag pinagsama-sama ang mga sample na ito sa minibatches, kailangan nating maglagay ng padding.

Magagawa ito gamit ang parehong teknik ng pagbibigay ng `collate_fn` function sa datasource:


In [3]:
def padify(b):
    # b is the list of tuples of length batch_size
    #   - first element of a tuple = label, 
    #   - second = feature (text sequence)
    # build vectorized sequence
    v = [encode(x[1]) for x in b]
    # first, compute max length of a sequence in this minibatch
    l = max(map(len,v))
    return ( # tuple of two tensors - labels and features
        torch.LongTensor([t[0]-1 for t in b]),
        torch.stack([torch.nn.functional.pad(torch.tensor(t),(0,l-len(t)),mode='constant',value=0) for t in v])
    )

train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=16, collate_fn=padify, shuffle=True)

### Pagsasanay ng embedding classifier

Ngayon na naitakda na natin ang tamang dataloader, maaari na nating sanayin ang modelo gamit ang training function na itinakda natin sa nakaraang unit:


In [4]:
net = EmbedClassifier(vocab_size,32,len(classes)).to(device)
train_epoch(net,train_loader, lr=1, epoch_size=25000)

3200: acc=0.6415625
6400: acc=0.6865625
9600: acc=0.7103125
12800: acc=0.726953125
16000: acc=0.739375
19200: acc=0.75046875
22400: acc=0.7572321428571429


(0.889799795315499, 0.7623160588611644)

> **Tandaan**: Nagsasanay lamang tayo para sa 25k na tala dito (mas mababa sa isang buong epoch) para makatipid sa oras, ngunit maaari mong ipagpatuloy ang pagsasanay, magsulat ng isang function para magsanay sa ilang mga epoch, at mag-eksperimento sa learning rate parameter upang makamit ang mas mataas na katumpakan. Dapat mong maabot ang katumpakan na humigit-kumulang 90%.


### EmbeddingBag Layer at Representasyon ng Variable-Length Sequence

Sa nakaraang arkitektura, kinakailangan nating i-pad ang lahat ng sequence upang magkapareho ang haba para magkasya ang mga ito sa isang minibatch. Hindi ito ang pinakaepektibong paraan upang i-representa ang mga sequence na may iba't ibang haba - isang alternatibong paraan ay ang paggamit ng **offset** vector, na maglalaman ng mga offset ng lahat ng sequence na nakaimbak sa isang malaking vector.

![Larawan na nagpapakita ng offset sequence representation](../../../../../translated_images/tl/offset-sequence-representation.eb73fcefb29b46ee.webp)

> **Note**: Sa larawan sa itaas, ipinapakita namin ang isang sequence ng mga karakter, ngunit sa ating halimbawa, nagtatrabaho tayo sa mga sequence ng salita. Gayunpaman, ang pangkalahatang prinsipyo ng pagre-representa ng mga sequence gamit ang offset vector ay nananatiling pareho.

Upang magtrabaho gamit ang offset representation, ginagamit natin ang [`EmbeddingBag`](https://pytorch.org/docs/stable/generated/torch.nn.EmbeddingBag.html) layer. Katulad ito ng `Embedding`, ngunit tumatanggap ito ng content vector at offset vector bilang input, at kasama rin nito ang averaging layer, na maaaring `mean`, `sum`, o `max`.

Narito ang binagong network na gumagamit ng `EmbeddingBag`:


In [5]:
class EmbedClassifier(torch.nn.Module):
    def __init__(self, vocab_size, embed_dim, num_class):
        super().__init__()
        self.embedding = torch.nn.EmbeddingBag(vocab_size, embed_dim)
        self.fc = torch.nn.Linear(embed_dim, num_class)

    def forward(self, text, off):
        x = self.embedding(text, off)
        return self.fc(x)

Upang ihanda ang dataset para sa pagsasanay, kailangan nating magbigay ng conversion function na maghahanda sa offset vector:


In [6]:
def offsetify(b):
    # first, compute data tensor from all sequences
    x = [torch.tensor(encode(t[1])) for t in b]
    # now, compute the offsets by accumulating the tensor of sequence lengths
    o = [0] + [len(t) for t in x]
    o = torch.tensor(o[:-1]).cumsum(dim=0)
    return ( 
        torch.LongTensor([t[0]-1 for t in b]), # labels
        torch.cat(x), # text 
        o
    )

train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=16, collate_fn=offsetify, shuffle=True)

Tandaan, na hindi tulad ng lahat ng naunang halimbawa, ang ating network ngayon ay tumatanggap ng dalawang parameter: data vector at offset vector, na may magkaibang sukat. Gayundin, ang ating data loader ay nagbibigay sa atin ng 3 halaga sa halip na 2: parehong text at offset vectors ay ibinibigay bilang mga tampok. Samakatuwid, kailangan nating bahagyang ayusin ang ating training function upang maasikaso ito:


In [7]:
net = EmbedClassifier(vocab_size,32,len(classes)).to(device)

def train_epoch_emb(net,dataloader,lr=0.01,optimizer=None,loss_fn = torch.nn.CrossEntropyLoss(),epoch_size=None, report_freq=200):
    optimizer = optimizer or torch.optim.Adam(net.parameters(),lr=lr)
    loss_fn = loss_fn.to(device)
    net.train()
    total_loss,acc,count,i = 0,0,0,0
    for labels,text,off in dataloader:
        optimizer.zero_grad()
        labels,text,off = labels.to(device), text.to(device), off.to(device)
        out = net(text, off)
        loss = loss_fn(out,labels) #cross_entropy(out,labels)
        loss.backward()
        optimizer.step()
        total_loss+=loss
        _,predicted = torch.max(out,1)
        acc+=(predicted==labels).sum()
        count+=len(labels)
        i+=1
        if i%report_freq==0:
            print(f"{count}: acc={acc.item()/count}")
        if epoch_size and count>epoch_size:
            break
    return total_loss.item()/count, acc.item()/count


train_epoch_emb(net,train_loader, lr=4, epoch_size=25000)

3200: acc=0.6153125
6400: acc=0.6615625
9600: acc=0.6932291666666667
12800: acc=0.715078125
16000: acc=0.7270625
19200: acc=0.7382291666666667
22400: acc=0.7486160714285715


(22.771553103007037, 0.7551983365323096)

## Semantic Embeddings: Word2Vec

Sa ating nakaraang halimbawa, natutunan ng model embedding layer na i-map ang mga salita sa vector representation, ngunit ang representasyong ito ay walang gaanong semantikong kahulugan. Maganda sana kung makakagawa tayo ng ganitong uri ng vector representation, kung saan ang mga magkatulad na salita o mga kasingkahulugan ay magkakaroon ng mga vector na malapit sa isa't isa batay sa ilang uri ng distansya ng vector (hal. euclidian distance).

Upang magawa ito, kailangan nating i-pre-train ang ating embedding model sa isang malaking koleksyon ng teksto sa isang partikular na paraan. Isa sa mga unang paraan upang sanayin ang semantic embeddings ay tinatawag na [Word2Vec](https://en.wikipedia.org/wiki/Word2vec). Ito ay nakabatay sa dalawang pangunahing arkitektura na ginagamit upang makabuo ng distributed representation ng mga salita:

 - **Continuous bag-of-words** (CBoW) — sa arkitekturang ito, sinasanay natin ang modelo upang hulaan ang isang salita mula sa nakapaligid na konteksto. Ibinigay ang ngram $(W_{-2},W_{-1},W_0,W_1,W_2)$, ang layunin ng modelo ay hulaan ang $W_0$ mula sa $(W_{-2},W_{-1},W_1,W_2)$.
 - **Continuous skip-gram** — kabaligtaran ng CBoW. Ginagamit ng modelo ang nakapaligid na window ng mga salita sa konteksto upang hulaan ang kasalukuyang salita.

Mas mabilis ang CBoW, habang ang skip-gram ay mas mabagal, ngunit mas mahusay sa pag-representa ng mga bihirang salita.

![Larawan na nagpapakita ng parehong CBoW at Skip-Gram na mga algorithm para i-convert ang mga salita sa mga vector.](../../../../../translated_images/tl/example-algorithms-for-converting-words-to-vectors.fbe9207a726922f6.webp)

Upang mag-eksperimento gamit ang word2vec embedding na pre-trained sa Google News dataset, maaari nating gamitin ang **gensim** library. Sa ibaba, makikita natin ang mga salitang pinakamalapit sa 'neural'.

> **Note:** Kapag unang gumawa ng word vectors, maaaring tumagal ng kaunting oras ang pag-download ng mga ito!


In [8]:
import gensim.downloader as api
w2v = api.load('word2vec-google-news-300')

In [9]:
for w,p in w2v.most_similar('neural'):
    print(f"{w} -> {p}")

neuronal -> 0.7804799675941467
neurons -> 0.7326500415802002
neural_circuits -> 0.7252851724624634
neuron -> 0.7174385190010071
cortical -> 0.6941086649894714
brain_circuitry -> 0.6923246383666992
synaptic -> 0.6699118614196777
neural_circuitry -> 0.6638563275337219
neurochemical -> 0.6555314064025879
neuronal_activity -> 0.6531826257705688


Maaari rin nating kalkulahin ang vector embeddings mula sa salita, na gagamitin sa pagsasanay ng classification model (ipinapakita lamang namin ang unang 20 na bahagi ng vector para sa kalinawan):


In [10]:
w2v.word_vec('play')[:20]

array([ 0.01226807,  0.06225586,  0.10693359,  0.05810547,  0.23828125,
        0.03686523,  0.05151367, -0.20703125,  0.01989746,  0.10058594,
       -0.03759766, -0.1015625 , -0.15820312, -0.08105469, -0.0390625 ,
       -0.05053711,  0.16015625,  0.2578125 ,  0.10058594, -0.25976562],
      dtype=float32)

Ang magandang bagay tungkol sa semantical embeddings ay maaari mong manipulahin ang vector encoding upang baguhin ang semantika. Halimbawa, maaari tayong magtanong upang makahanap ng isang salita, na ang vector representation ay magiging kasing lapit hangga't maaari sa mga salitang *hari* at *babae*, at kasing layo hangga't maaari mula sa salitang *lalaki*:


In [10]:
w2v.most_similar(positive=['king','woman'],negative=['man'])[0]

('queen', 0.7118192911148071)

Parehong CBoW at Skip-Grams ay mga “predictive” embeddings, dahil tanging mga lokal na konteksto lamang ang isinasaalang-alang nila. Ang Word2Vec ay hindi gumagamit ng global na konteksto.

**FastText**, ay nakabatay sa Word2Vec sa pamamagitan ng pag-aaral ng mga vector na representasyon para sa bawat salita at ang mga character n-grams na matatagpuan sa loob ng bawat salita. Ang mga halaga ng mga representasyon ay pagkatapos ay ina-average sa isang vector sa bawat hakbang ng pagsasanay. Bagama't nagdadagdag ito ng maraming karagdagang pagkalkula sa pre-training, pinapahintulutan nito ang word embeddings na mag-encode ng impormasyon sa sub-word.

Isa pang pamamaraan, ang **GloVe**, ay gumagamit ng ideya ng co-occurrence matrix, at gumagamit ng mga neural na pamamaraan upang i-decompose ang co-occurrence matrix sa mas ekspresibo at hindi linear na mga word vector.

Maaari mong subukan ang halimbawa sa pamamagitan ng pagpapalit ng embeddings sa FastText at GloVe, dahil sinusuportahan ng gensim ang iba't ibang mga modelo ng word embedding.


## Paggamit ng Pre-Trained Embeddings sa PyTorch

Maaari nating baguhin ang halimbawa sa itaas upang punan ang matrix sa ating embedding layer gamit ang semantikal na embeddings, tulad ng Word2Vec. Kailangan nating isaalang-alang na ang mga bokabularyo ng pre-trained embedding at ng ating text corpus ay malamang na hindi magtutugma, kaya i-initialize natin ang mga weights para sa mga nawawalang salita gamit ang random na mga halaga:


In [11]:
embed_size = len(w2v.get_vector('hello'))
print(f'Embedding size: {embed_size}')

net = EmbedClassifier(vocab_size,embed_size,len(classes))

print('Populating matrix, this will take some time...',end='')
found, not_found = 0,0
for i,w in enumerate(vocab.get_itos()):
    try:
        net.embedding.weight[i].data = torch.tensor(w2v.get_vector(w))
        found+=1
    except:
        net.embedding.weight[i].data = torch.normal(0.0,1.0,(embed_size,))
        not_found+=1

print(f"Done, found {found} words, {not_found} words missing")
net = net.to(device)

Embedding size: 300
Populating matrix, this will take some time...Done, found 41080 words, 54732 words missing


Ngayon, mag-ensayo tayo ng ating modelo. Tandaan na ang oras na kinakailangan upang mag-ensayo ng modelo ay mas mahaba kumpara sa nakaraang halimbawa, dahil sa mas malaking sukat ng embedding layer, at sa gayon mas mataas na bilang ng mga parameter. Gayundin, dahil dito, maaaring kailanganin nating mag-ensayo ng ating modelo sa mas maraming halimbawa kung nais nating maiwasan ang overfitting.


In [12]:
train_epoch_emb(net,train_loader, lr=4, epoch_size=25000)

3200: acc=0.6359375
6400: acc=0.68109375
9600: acc=0.7067708333333333
12800: acc=0.723671875
16000: acc=0.73625
19200: acc=0.7463541666666667
22400: acc=0.7560714285714286


(214.1013875559821, 0.7626759436980166)

Sa ating kaso, hindi natin nakikita ang malaking pagtaas sa accuracy, na malamang dahil sa magkaibang bokabularyo.  
Upang malampasan ang problema ng magkaibang bokabularyo, maaari nating gamitin ang isa sa mga sumusunod na solusyon:  
* Muling i-train ang word2vec model gamit ang ating bokabularyo  
* I-load ang ating dataset gamit ang bokabularyo mula sa pre-trained word2vec model. Ang bokabularyo na gagamitin para i-load ang dataset ay maaaring tukuyin habang naglo-load.  

Ang huling paraan ay mukhang mas madali, lalo na dahil ang PyTorch `torchtext` framework ay may built-in na suporta para sa embeddings. Halimbawa, maaari nating i-instantiate ang GloVe-based na bokabularyo sa ganitong paraan:  


In [14]:
vocab = torchtext.vocab.GloVe(name='6B', dim=50)

100%|█████████▉| 399999/400000 [00:15<00:00, 25411.14it/s]


Ang na-load na bokabularyo ay may mga sumusunod na pangunahing operasyon:
* Ang `vocab.stoi` na diksyunaryo ay nagbibigay-daan sa atin na i-convert ang salita sa index nito sa diksyunaryo
* Ang `vocab.itos` ay gumagawa ng kabaligtaran - kino-convert ang numero sa salita
* Ang `vocab.vectors` ay ang array ng embedding vectors, kaya upang makuha ang embedding ng isang salita na `s`, kailangan nating gamitin ang `vocab.vectors[vocab.stoi[s]]`

Narito ang halimbawa ng pag-manipula ng embeddings upang ipakita ang equation **kind-man+woman = queen** (Kinailangan kong ayusin nang kaunti ang coefficient para gumana ito):


In [15]:
# get the vector corresponding to kind-man+woman
qvec = vocab.vectors[vocab.stoi['king']]-vocab.vectors[vocab.stoi['man']]+1.3*vocab.vectors[vocab.stoi['woman']]
# find the index of the closest embedding vector 
d = torch.sum((vocab.vectors-qvec)**2,dim=1)
min_idx = torch.argmin(d)
# find the corresponding word
vocab.itos[min_idx]

'queen'

Upang sanayin ang classifier gamit ang mga embeddings na iyon, kailangan muna nating i-encode ang ating dataset gamit ang GloVe vocabulary:


In [16]:
def offsetify(b):
    # first, compute data tensor from all sequences
    x = [torch.tensor(encode(t[1],voc=vocab)) for t in b] # pass the instance of vocab to encode function!
    # now, compute the offsets by accumulating the tensor of sequence lengths
    o = [0] + [len(t) for t in x]
    o = torch.tensor(o[:-1]).cumsum(dim=0)
    return ( 
        torch.LongTensor([t[0]-1 for t in b]), # labels
        torch.cat(x), # text 
        o
    )

Tulad ng nakita natin sa itaas, lahat ng vector embeddings ay nakaimbak sa `vocab.vectors` matrix. Ginagawa nitong napakadaling i-load ang mga weights na iyon sa weights ng embedding layer gamit ang simpleng pagkopya:


In [17]:
net = EmbedClassifier(len(vocab),len(vocab.vectors[0]),len(classes))
net.embedding.weight.data = vocab.vectors
net = net.to(device)

In [18]:
train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=16, collate_fn=offsetify, shuffle=True)
train_epoch_emb(net,train_loader, lr=4, epoch_size=25000)

3200: acc=0.6271875
6400: acc=0.68078125
9600: acc=0.7030208333333333
12800: acc=0.71984375
16000: acc=0.7346875
19200: acc=0.7455729166666667
22400: acc=0.7529464285714286


(35.53972978646833, 0.7575175943698017)

Isa sa mga dahilan kung bakit hindi natin nakikita ang makabuluhang pagtaas sa katumpakan ay dahil sa ang ilang mga salita mula sa ating dataset ay nawawala sa pre-trained na GloVe vocabulary, at dahil dito, sila ay karaniwang hindi pinapansin. Upang malampasan ang katotohanang ito, maaari tayong mag-train ng sarili nating embeddings sa ating dataset.


## Mga Kontekstwal na Embeddings

Isang pangunahing limitasyon ng tradisyunal na pretrained embedding representations tulad ng Word2Vec ay ang problema ng word sense disambiguation. Bagama't ang pretrained embeddings ay maaaring makuha ang ilang kahulugan ng mga salita sa konteksto, ang bawat posibleng kahulugan ng isang salita ay naka-encode sa parehong embedding. Ito ay maaaring magdulot ng problema sa mga downstream models, dahil maraming salita, tulad ng salitang 'play', ay may iba't ibang kahulugan depende sa konteksto kung saan ito ginagamit.

Halimbawa, ang salitang 'play' sa dalawang magkaibang pangungusap ay may magkaibang kahulugan:
- Pumunta ako sa isang **play** sa teatro.
- Gusto ni John na **maglaro** kasama ang kanyang mga kaibigan.

Ang pretrained embeddings sa itaas ay kumakatawan sa parehong mga kahulugan ng salitang 'play' sa iisang embedding. Upang malampasan ang limitasyong ito, kailangan nating bumuo ng embeddings batay sa **language model**, na sinanay sa isang malaking corpus ng teksto, at *alam* kung paano maaaring pagsamahin ang mga salita sa iba't ibang konteksto. Ang talakayan tungkol sa kontekstwal na embeddings ay labas sa saklaw ng tutorial na ito, ngunit babalikan natin ito kapag pinag-uusapan ang mga language models sa susunod na unit.



---

**Paunawa**:  
Ang dokumentong ito ay isinalin gamit ang AI translation service na [Co-op Translator](https://github.com/Azure/co-op-translator). Bagama't sinisikap naming maging tumpak, tandaan na ang mga awtomatikong pagsasalin ay maaaring maglaman ng mga pagkakamali o hindi pagkakatugma. Ang orihinal na dokumento sa kanyang katutubong wika ang dapat ituring na opisyal na sanggunian. Para sa mahalagang impormasyon, inirerekomenda ang propesyonal na pagsasalin ng tao. Hindi kami mananagot sa anumang hindi pagkakaunawaan o maling interpretasyon na maaaring magmula sa paggamit ng pagsasaling ito.
